In [1]:
import os
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
# 조사기준 → 품목(대분류) 매핑 딕셔너리 예시
ITEM_CATEGORY_MAP = {
    "청풍명월(20kg)": "쌀",
    "일반정미(20kg)": "쌀",
    "국산, 백태 1kg": "콩",
    "국산 500g": "참깨",
    "수입산, 500g": "참깨",
    "부사 400g1개, 중품": "사과",
    "신고배 600g1개, 중품": "배",
    "100g 정도 10개": "감귤",
    "굵은것 1kg": "밤",
    "통무우 2kg": "무",
    "통배추 2kg": "배추",
    "국산, 400g": "고추가루",
    "망에든것 1kg": "양파",
    "1kg": "대파",  # 주의: "1kg"만으로 구분하기 애매하면 수정 필요
    "국산, 중품, 100g": "깐마늘",
    "수입콩, 포장, 300g": "콩나물",
    "한우등심1등급 600g": "쇠고기",
    "국산, 삼겹살 600g": "돼지고기",
    "하림, 육계 1마리 1kg": "생닭",
    "대란 1판(30개)": "달걀",
    "수입, 25cm정도, 1마리": "조기",
    "수입, 냉동40cm1마리": "동태",
    "25cm정도1마리": "오징어(냉동)",
    "재래생김 1속, 중품": "김",
    "멸치(국물용) 500g": "멸치",
    "제일제당중력분 1kg": "밀가루",
    "제일제당정백당 1kg": "설탕",
    "국산콩, 풀무원(부침용), 300g": "두부",
    "수입콩 (부침용) 300g": "두부",
    "신라면 1봉지": "라면",
    "백설표 1.8L 대두유": "식용유",
    "오뚜기320ml 1병": "참기름",
    "동서맥심175g 1병": "분말커피",
    "O2린 360ml, 1병": "소주",
    "카스 500ml 1병": "맥주",
    "백화수복, 1.8L 1병": "청주",
    "크리넥스 35mⅹ24롤(백색)": "화장지",
    "삼립 식빵 750g": "빵",
    "매일우유 1000L": "우유",
    "남양임페리얼 3단계": "분유",
    "애경 스파크 3kg": "세제",
}

## Part 1: **원본 엑셀 파일 → 월별 통합 엑셀 파일**

**설명:**  
 - 지정된 폴더 내의 엑셀 파일들은 파일명이 "년도-월" (예: 2014-7.xlsx) 형태입니다.  
 - 각 엑셀 파일에는 **4행**(행정구역 동 이름)과 **5행**(마트 이름)이 헤더로 들어가 있으며, **6행부터** 실제 데이터(가격)가 시작됩니다.  
 - 각 시트에는 **첫 번째 열**에 품목(예: 생필품)이 있으며, 이후 열은 **다중 헤더**(동, 마트)로 구성되어 있습니다.  
 - 동일 동에 여러 마트가 있을 경우(또는 여러 시트를 선택한 경우) **모든 가격의 평균**을 산출합니다.

 **구현 방식:**  
 1. 지정한 시트별로 데이터를 읽은 후, `melt`를 통해 **롱 포맷(long format)**으로 변환  
 2. 품목과 동별로 그룹화하여 **가격의 평균**을 구하고  
 3. 이를 **pivot**하여 다시 와이드 포맷(wide format)으로 변환합니다.

In [3]:
def process_excel_file(file_path, sheet_list):
    """
    [개별 엑셀 → '동'별 평균 가격]을 추출하여
    (행: 조사기준, 열: 동, 값: 가격평균) 형태의 DF를 반환합니다.
    
    ※ 주요 변경점
      - C열(3번째 열, index=2)을 '조사기준'으로 삼고,
      - 5번째 열 이후(index>=4)부터 실제 동별 가격 데이터로 활용
      - 가격=0인 경우 평균에서 제외
    """
    combined_long = []

    for sheet in sheet_list:
        try:
            # 1) 엑셀 읽기 (4행+5행을 다중 헤더)
            df = pd.read_excel(file_path, sheet_name=sheet, header=[3, 4])
            
            # 2) 다중 헤더를 평탄화
            df.columns = [
                "_".join([str(x).strip() for x in col]).strip()
                for col in df.columns
            ]
            
            # 3) C열(index=2) + 5번째 열 이후(index>=4)만 사용
            if df.shape[1] < 5:
                print(f"[경고] 시트 '{sheet}' → 열 수 부족: {os.path.basename(file_path)}")
                continue
            
            # 조사기준(=C열) + 동별 가격 정보 부분만 추출
            df = df.iloc[:, [2] + list(range(4, df.shape[1]))]
            
            # 첫 컬럼명을 '조사기준'으로 변경
            df.rename(columns={df.columns[0]: '조사기준'}, inplace=True)
            
            # 병합된 셀이 풀려도 조사기준이 비어있을 수 있으므로 ffill
            df['조사기준'] = df['조사기준'].ffill()
            
            # 4) melt로 '조사기준' (id_vars) + 나머지(동+마트) 구조로 변환
            df_melt = df.melt(
                id_vars=['조사기준'],
                var_name='동_마트',
                value_name='가격'
            )
            
            # 5) '동_마트'를 '동'과 '마트'로 분리
            df_melt[['동', '마트']] = df_melt['동_마트'].str.split('_', n=1, expand=True)
            df_melt.drop(columns=['동_마트'], inplace=True)
            
            # 6) 가격이 숫자형이 아니면 NaN 처리 후 제거
            df_melt['가격'] = pd.to_numeric(df_melt['가격'], errors='coerce')
            df_melt.dropna(subset=['가격'], inplace=True)
            
            # 7) 가격=0은 ‘데이터 없음’으로 간주하여 제외
            df_melt = df_melt[df_melt['가격'] != 0]
            
            combined_long.append(df_melt)

        except Exception as e:
            print(f"[오류] 시트 '{sheet}' 처리 실패 ({os.path.basename(file_path)}): {e}")

    # 모든 시트에서 수집된 데이터가 없으면 None 리턴
    if not combined_long:
        return None

    # 8) 여러 시트의 데이터를 수직결합 → (조사기준, 동) 그룹으로 가격 평균
    df_all = pd.concat(combined_long, ignore_index=True)
    df_grouped = df_all.groupby(['조사기준', '동'], as_index=False)['가격'].mean()

    # 9) 피벗 (행: 조사기준, 열: 동, 값: 평균 가격)
    df_pivot = df_grouped.pivot(index='조사기준', columns='동', values='가격').reset_index()

    return df_pivot


## Part 2: **월별 통합 엑셀 파일 → 최종 통합 엑셀 파일**

**설명:**  
- Part 1에서 생성된 **월별 통합 엑셀 파일들**을 읽어와  
- 각 파일의 데이터를 **수직으로 결합(append)**하여 하나의 최종 엑셀 파일을 생성합니다.  
- 파일명에서 월 정보를 추출하여 새 컬럼(예: '월')에 추가할 수 있습니다.

In [4]:
def process_folder_to_monthly(folder_path, sheet_list, output_folder):
    """
    [폴더 내 모든 .xlsx] 파일에 대해 process_excel_file()을 수행,
    각 결과를 월별(파일별)로 따로 저장합니다.
    
    :param folder_path: 원본 엑셀 폴더 경로
    :param sheet_list: 처리할 시트명 리스트(e.g. ['SSM','대형마트'])
    :param output_folder: 결과(월별 .xlsx)를 저장할 폴더
    """
    file_list = glob.glob(os.path.join(folder_path, "*.xlsx"))
    
    for file in tqdm(file_list, desc="원본 파일 처리중"):
        df_result = process_excel_file(file, sheet_list)
        if df_result is None:
            continue
        
        base_name = os.path.splitext(os.path.basename(file))[0]
        output_path = os.path.join(output_folder, f"{base_name}_processed.xlsx")
        
        # 피벗결과 df_result는 [조사기준, 동1, 동2, …] 형태
        df_result.to_excel(output_path, index=False)
        print(f"[저장 완료] {output_path}")


In [5]:
def combine_monthly_files(input_folder, final_output_file):
    """
    *_processed.xlsx들을 모아 최종 통합 파일을 만드는 함수.
    파일명에서 '월' 정보를 추출해 첫 컬럼으로 붙이고,
    '조사기준'을 바탕으로 '품목'을 생성한 뒤,
    컬럼을 [월, 품목, 조사기준, 나머지 동들] 순으로 재배치한다.
    """
    # 처리 대상 파일 목록
    file_list = glob.glob(os.path.join(input_folder, "*_processed.xlsx"))
    monthly_dfs = []
    
    for file in tqdm(file_list, desc="월별 파일 결합중"):
        # 파일명에서 월(예: "2014-7") 추출
        base_name = os.path.splitext(os.path.basename(file))[0]
        month_info = base_name.replace("_processed", "")
        
        # 파일 로드
        df = pd.read_excel(file)
        
        # 월 컬럼을 가장 왼쪽에 추가
        df.insert(0, "월", month_info)
        
        # 조사기준 → 품목 변환, 없으면 "기타"
        df["품목"] = df["조사기준"].map(ITEM_CATEGORY_MAP).fillna("기타")
        
        # 컬럼 순서를 "월, 품목, 조사기준, 나머지 동들"로 재배치
        current_cols = df.columns.tolist()
        base_cols = ["월", "조사기준", "품목"]
        other_cols = [c for c in current_cols if c not in base_cols]
        final_col_order = ["월", "품목", "조사기준"] + other_cols
        df = df[final_col_order]
        
        monthly_dfs.append(df)
    
    # 모든 월별 파일을 수직 결합 후 저장
    if monthly_dfs:
        final_df = pd.concat(monthly_dfs, ignore_index=True)
        final_df.to_excel(final_output_file, index=False)
        print(f"[최종 파일 저장 완료] {final_output_file}")
    else:
        print("[주의] 통합할 월별 파일이 없습니다.")


## 메인 실행 코드

In [6]:
if __name__ == "__main__":
    # 1) 원본 엑셀 폴더
    original_folder = "C:\\Users\\administration\\Desktop\\origin_data"
    
    # 2) 월별 결과 저장 폴더
    monthly_output_folder = "C:\\Users\\administration\\Desktop\\1st_convert"
    
    # 3) 최종 통합 엑셀 파일 경로
    final_output_file = "C:\\Users\\administration\\Desktop\\2nd_convert\\2017_대전.xlsx"
    
    # 4) 처리할 시트 리스트
    sheets_to_process = ['SSM', '대형마트']
    
    # 폴더 미리 생성
    os.makedirs(monthly_output_folder, exist_ok=True)
    os.makedirs(os.path.dirname(final_output_file), exist_ok=True)
    
    # Part 1) 폴더 내 모든 엑셀(월별)을 순회 처리 → *_processed.xlsx 생성
    process_folder_to_monthly(original_folder, sheets_to_process, monthly_output_folder)
    
    # Part 2) *_processed.xlsx들을 하나로 통합해 최종 파일 생성
    combine_monthly_files(monthly_output_folder, final_output_file)


원본 파일 처리중:  67%|██████▋   | 2/3 [00:00<00:00,  3.69it/s]

[저장 완료] C:\Users\administration\Desktop\1st_convert\2017-1_processed.xlsx
[저장 완료] C:\Users\administration\Desktop\1st_convert\2017-2_processed.xlsx


원본 파일 처리중: 100%|██████████| 3/3 [00:00<00:00,  3.96it/s]


[저장 완료] C:\Users\administration\Desktop\1st_convert\2017-3_processed.xlsx


월별 파일 결합중: 100%|██████████| 33/33 [00:01<00:00, 31.84it/s]


[최종 파일 저장 완료] C:\Users\administration\Desktop\2nd_convert\2017_대전.xlsx
